In [4]:
import requests
import pandas

import datetime_translator

brandIds = { 202, 88, 31, 123, 101, 122, 36, 48, 135 }
data = {}

for brandId in brandIds:
    headers = { 'User-Agent': '', 'content-type':  'application/json' }
    jsonData = '{"variables":{"area":"salt-lake","brandId":%d,"countryCode":"US","criteria":{"location_type":"county"},"fuel":1,"maxAge":0,"regionCode":"UT"},"query":"query LocationByArea($area: String, $brandId: Int, $countryCode: String, $criteria: Criteria, $fuel: Int, $maxAge: Int, $regionCode: String) { locationByArea( area: $area countryCode: $countryCode criteria: $criteria regionCode: $regionCode ) { displayName locationType stations(brandId: $brandId, fuel: $fuel, maxAge: $maxAge) { results { address { country line1 line2 locality postalCode region  } brands { brand_id branding_type image_url name  } latitude longitude fuels id name prices(fuel: $fuel) { cash { nickname posted_time price  } credit { nickname posted_time price  } discount fuel_product } } } } }"}' %(brandId)
    #jsonData = '{"variables":{"area":"davis","brandId":%d,"countryCode":"US","criteria":{"location_type":"county"},"fuel":1,"maxAge":0,"regionCode":"UT"},"query":"query LocationByArea($area: String, $brandId: Int, $countryCode: String, $criteria: Criteria, $fuel: Int, $maxAge: Int, $regionCode: String) { locationByArea( area: $area countryCode: $countryCode criteria: $criteria regionCode: $regionCode ) { displayName locationType stations(brandId: $brandId, fuel: $fuel, maxAge: $maxAge) { results { address { country line1 line2 locality postalCode region  } brands { brand_id branding_type image_url name  } latitude longitude fuels id name prices(fuel: $fuel) { cash { nickname posted_time price  } credit { nickname posted_time price  } discount fuel_product } } } } }"}' %(brandId)
    response = requests.post('https://www.gasbuddy.com/graphql', headers=headers, data=jsonData)
    jsonResponse = response.json()
    stations = jsonResponse['data']['locationByArea']['stations']

    for station in stations['results']:
        if int(station['brands'][0]['brand_id']) in brandIds and (station['prices'][0]['credit']['price'] != 0):
            stationId = station['id']
            data[stationId] = {
                'StationName': station['name'],
                'BrandName': station['brands'][0]['name'],
                'AddressLine1': station['address']['line1'],
                'City': station['address']['locality'],
                'RegularFuelPrice': '${:.2f}'.format(station['prices'][0]['credit']['price']),
                'TimeSinceReported': datetime_translator.translate(station['prices'][0]['credit']['posted_time']),
                'ReportedBy': station['prices'][0]['credit']['nickname']
            }

output = pandas.DataFrame(data)
output.transpose().sort_values(by='RegularFuelPrice')

,StationName,BrandName,AddressLine1,City,RegularFuelPrice,TimeSinceReported,ReportedBy
201795,Chevron,Chevron,2474 S 7200 W,Magna,$3.69,1 day,joyluck23
83380,Chevron,Chevron,7198 W 3500 S,Magna,$3.69,1 day,JulieeJayy
131692,Holiday,Holiday Oil (UT),2644 S 8400 W,Magna,$3.69,1 day,Rainneyday
194281,Holiday,Holiday Oil (UT),3584 S 8400 W,Magna,$3.71,9 hours,andralaine
68385,Exxon,Exxon,7210 W 3500 S,Magna,$3.71,1 day,JulieeJayy
128320,Maverik,Maverik,2678 South 5600 West,West Valley,$3.79,21 minutes,Owner
201340,Maverik,Maverik,5969 West 4100 South,West Valley City,$3.79,21 minutes,Owner
62566,Maverik,Maverik,1927 S. 3230 W.,Salt Lake City,$3.79,21 minutes,Owner
79971,Maverik,Maverik,5610 WEST 5400 SOUTH,West Valley,$3.79,21 minutes,Owner
159136,Maverik,Maverik,1445 S. Gustin Road,Salt Lake City,$3.79,21 minutes,Owner
